In [1]:
!pip install openai==0.28

In [2]:
pip install fuzzywuzzy[speedup]


Note: you may need to restart the kernel to use updated packages.


In [3]:
def extract_phone_name(user_input, phones):
    user_input_words = user_input.lower().split()
    
    # Iterate through each phone name
    for phone in phones:
        phone_words = phone.lower().split()
        
        # Check if any of the user input words are in the phone name words
        if any(word in phone_words for word in user_input_words):
            return phone
    
    # If no match, return None
    return None


In [1]:
import openai
import pandas as pd
from fuzzywuzzy import fuzz

# Set your OpenAI API key
openai.api_key = 'sk-proj-kA9GfyXjrqh9lgY2ppr1T3BlbkFJxw1veilvpmfOhrvWocMT'

def read_feedback_data(file_path):
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except pd.errors.EmptyDataError:
        print("The CSV file is empty.")
        return None

# Fuzzy matching function for phone names
def extract_phone_name(user_input, phones):
    user_input_lower = user_input.lower()
    threshold = 70  # Adjust the threshold for matching sensitivity
    best_match = None
    best_score = 0
    
    for phone in phones:
        score = fuzz.partial_ratio(user_input_lower, phone.lower())
        if score > threshold and score > best_score:
            best_match = phone
            best_score = score
    
    return best_match

def get_summarization(phone_name):
    if not phone_name:
        return 'No phone selected'
    
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',  # or 'gpt-4'
            messages=[
                {"role": "system", "content": "You are a helpful assistant who summarizes reviews for each phone and give me summary and 3 andvantages"},
                {"role": "user", "content": f"Summarize the reviews for the phone: {phone_name}."}
            ]
        )
        
        # Log the full response for debugging purposes
       
        
        summarization = response['choices'][0]['message']['content']
        return summarization
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return f"An error occurred during summarization: {str(e)}"

def ai_assistant(user_input, phones):
    phone_name = extract_phone_name(user_input, phones)
    summarization = get_summarization(phone_name)
    return {"Summary": summarization, "Phone Name": phone_name}

def display_result(result):
    if result.get("Phone Name") and result.get("Summary"):
        # Pretty print the result
        print("\n" + "="*50)
        print(f"Phone Name: {result['Phone Name']}")
        print("="*50)
        print("Summary:")
        print("-" * 50)
        print(f"{result['Summary']}")
        print("=" * 50)
    else:
        print("No valid phone name found or no summary generated.")

# Main execution
user_input = input("How can I help you? ")
file_path = "phone info (1).csv"  # Update with your actual file path
feedback_data = read_feedback_data(file_path)

if feedback_data is not None:
    phones = feedback_data['Title'].unique()  # Adjust 'Title' to your actual column name
    result = ai_assistant(user_input, phones)
    display_result(result)
else:
    print("Could not read feedback data.")

How can I help you?  iphone 13



Phone Name: iPhone 13 - 128GB - Network Locked - Pre-Owned Very Good Condition
Summary:
--------------------------------------------------
Overall, the reviews for the iPhone 13 - 128GB - Network Locked - Pre-Owned Very Good Condition are positive. Users have praised its high-quality camera, fast performance, and sleek design. Some customers have noted that the network lock can be a drawback for those looking to switch carriers. 

Advantages:
1. High-quality camera
2. Fast performance
3. Sleek design
